### Attention

`pytesseract` needs a tesseract installation to work. `pdf2image` requires a poppler installation. Here are starting links for the two installations:

**Tesseract**: https://github.com/UB-Mannheim/tesseract/wiki
**pdf2image (poppler)**: https://pdf2image.readthedocs.io/en/latest/installation.html

Keep in mind that on windows systems you need to add the installations to PATH

# Setup

In [ ]:
from pdf2image import convert_from_path
from PIL import Image, ImageFilter
import pytesseract
import os

In [ ]:
def preprocess_image(image):
    """
    Apply image processing techniques to enhance OCR accuracy.

    Args:
        image (PIL.Image): Original image.

    Returns:
        PIL.Image: Preprocessed image.
    """
    # Convert to grayscale
    image = image.convert('L')
    # Apply adaptive thresholding for binarization
    image = image.point(lambda x: 0 if x < 128 else 255, '1')
    # Remove noise and smooth the image
    image = image.filter(ImageFilter.MedianFilter())
    return image

def merge_images(images):
    """
    Vertically merge a list of images into one single image.

    Args:
        images (list): List of PIL Image objects.

    Returns:
        PIL.Image: A single merged image.
    """
    preprocessed_images = [preprocess_image(img) for img in images]
    widths, heights = zip(*(i.size for i in preprocessed_images))
    total_height = sum(heights)
    max_width = max(widths)
    merged_image = Image.new('L', (max_width, total_height), 'white')
    y_offset = 0
    for im in preprocessed_images:
        merged_image.paste(im, (0, y_offset))
        y_offset += im.size[1]
    return merged_image


def extract_text_from_pdf(pdf_path, output_txt_path, dpi):
    """
    Extract text from a PDF file by converting it to a single image and then using OCR.

    Args:
    pdf_path (str): Path to the PDF file.
    output_txt_path (str): Path to the output text file to save the extracted text.
    """
    # Convert PDF to images (one per page)
    try:
        images = convert_from_path(pdf_path, dpi=dpi)
    except:
        images = convert_from_path(pdf_path, dpi=dpi-100)

    # Configuring tesseract to use a specific PSM and OEM
    custom_oem_psm_config = '--oem 3 --psm 3'

    # Use pytesseract to extract text
    text = ''
    for image in images:
        text += pytesseract.image_to_string(image, lang='deu', config=custom_oem_psm_config)

    # Save the extracted text to a file
    with open(output_txt_path, 'w', encoding='utf-8') as file:
        file.write(text)


# Parameters

In [ ]:
# Paths
PDF_DIRECTORY = 'data/pdfs'
TEXT_DIRECTORY = 'data/texts'

# DPI for PDF to image transformation
dpi = 300

In [ ]:
# Crate folders if the don't exist
if not os.path.exists(TEXT_DIRECTORY):
    os.makedirs(TEXT_DIRECTORY)

# Extract Text

In [ ]:
# List all pdfs to read
pdf_names = os.listdir(PDF_DIRECTORY)

# For each pdf create a txt file
for pdf_name in pdf_names:
    
    in_dir = f'{PDF_DIRECTORY}/{pdf_name}'
    file_name = pdf_name[:-4] + '.txt'
    txt_out = TEXT_DIRECTORY + '/' + file_name

    print(file_name)

    if file_name not in os.listdir(TEXT_DIRECTORY):
        extract_text_from_pdf(in_dir, txt_out, dpi)